In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-4
PRIOR_FLAG = False
ONLY_FORWARD = False
CUDA = torch.cuda.is_available()
PATH = 'ep-fb-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE, PRIOR_FLAG, ONLY_FORWARD)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
if PRIOR_FLAG:
    optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    models = (enc_eta, enc_z)
else:
    oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
    if CUDA:
        oneshot_eta.cuda().to(DEVICE)
    optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\1000 (30s),  symKL_DB_eta: 281.044,  symKL_DB_z: 217.368,  loss: -2359.028,  ess: 3.082,  kl_eta_ex: 601.233,  kl_eta_in: 136.702,  kl_z_ex: 146.994,  kl_z_in: 30.743
epoch: 1\1000 (29s),  symKL_DB_eta: 264.719,  symKL_DB_z: 209.816,  loss: -2274.210,  ess: 3.104,  kl_eta_ex: 573.482,  kl_eta_in: 134.630,  kl_z_ex: 143.548,  kl_z_in: 30.154
epoch: 2\1000 (29s),  symKL_DB_eta: 245.316,  symKL_DB_z: 200.890,  loss: -2174.056,  ess: 3.137,  kl_eta_ex: 537.884,  kl_eta_in: 131.629,  kl_z_ex: 138.037,  kl_z_in: 29.390
epoch: 3\1000 (41s),  symKL_DB_eta: 225.826,  symKL_DB_z: 189.327,  loss: -2070.670,  ess: 3.176,  kl_eta_ex: 495.398,  kl_eta_in: 128.074,  kl_z_ex: 131.601,  kl_z_in: 28.494
epoch: 4\1000 (44s),  symKL_DB_eta: 207.085,  symKL_DB_z: 175.865,  loss: -1972.396,  ess: 3.220,  kl_eta_ex: 452.969,  kl_eta_in: 124.423,  kl_z_ex: 121.750,  kl_z_in: 27.377
epoch: 5\1000 (44s),  symKL_DB_eta: 189.650,  symKL_DB_z: 162.641,  loss: -1880.230,  ess: 3.267,  kl_eta_ex: 412.707,  

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)

In [ ]:
BATCH_SIZE_TEST = 50
Model_Params_Test = (N, K, D, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST)
obs, metric_step, reused = test(Eubo_cfz_os_eta, Data, Train_Params, Model_Params_Test, models)
(q_eta, _, q_z, _, _, _) = reused

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap